# Data Cleaning 2: aka Machine Learning Data Prep

This file prepares two datasets so they can be directly input into the machine learning code.

Dataset 1: ML_Base_data --> this is a csv file that holds the traffic lines for our baseline data. The machine learning model can make predictions for these road segments.

Dataset 2: SampledData --> our initial dataset with all the sports data is way too large to work with efficiently, so we subset for easier use.


# Prepping Baseline Data

We want to have an easy dataset to be able to apply our machine learning model and show results on a map.

In [2]:
#import statements
import geopandas as gpd
import os
from datetime import datetime
from shapely.geometry import LineString
    
#read the shapefile
gdf = gpd.read_file(r"C:\Users\Logan\Desktop\School Files\GIS 5572\Final\BaseLines\no_traffic_lines.shp")

#calculate the 'Latitude' and 'Longitude' based on the center of the Line String
gdf['Latitude'] = gdf['geometry'].centroid.y
gdf['Longitude'] = gdf['geometry'].centroid.x
gdf['geometry_wkt'] = gdf['geometry'].to_wkt()
#calculate segment length and distance from the road line data
gdf['SegmentLength'] = gdf['FromCumu_2'] - gdf['ToCumul_Ki']
gdf['Distance'] = (gdf['FromCumu_2'] + gdf['ToCumul_Ki']) / 2

#select we want to be used
columns_to_keep = {
    'SourceOID': 'SourceOID',
    'Latitude': 'Latitude',
    'Longitude': 'Longitude',
    'geometry_wkt' : 'geometry_wkt',
    'SegmentLength' : 'SegmentLength',
    'Distance' : 'Distance'
}
gdf = gdf.rename(columns=columns_to_keep)[list(columns_to_keep.values())]

#save to CSV
gdf.to_csv(r"C:\Users\Logan\Desktop\School Files\GIS 5572\Final\ML_Base_data_new.csv", index=False)
print(f"Saved")

C:\Users\Logan\AppData\Local\Temp\ipykernel_12272\2895185673.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['Latitude'] = gdf['geometry'].centroid.y
C:\Users\Logan\AppData\Local\Temp\ipykernel_12272\2895185673.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['Longitude'] = gdf['geometry'].centroid.x


Saved


# Subsetting our full sporting event dataset

The full sporting dataset is very large, so we want to utilize a smaller size

In [ ]:
#imports
import os
import pandas as pd

#load the large dataset from the CSV file into a DataFrame
data = pd.read_csv(r"C:\Users\Logan\Desktop\School Files\GIS 5572\Final\AllTrafficCost3.csv")

#create a random sample of 20% of the data
sampled_data = data.sample(frac=0.2, random_state=42)

#save the sampled data to a new CSV file
sampled_data.to_csv('SampledData2.csv', index=False)